In [8]:
import pandas as pd
from prettytable import PrettyTable


def get_num_processed_data(data_path):
    with open(data_path, "r") as f:
        data = f.readlines()
    return len(data)

def get_num_templates(data_path):
    return len(pd.read_csv(data_path))

def get_residual_proportion(data_path):
    num_all_data = get_num_processed_data(data_path)
    with open(data_path + ".processed", "r") as f:
        num_processed = len(f.readlines())
    return "{:.2f}".format(num_processed / num_all_data * 100)


all_rxn_class = ["AdamsDecarboxylation",
                "Baylis-HillmanReaction",
                "Buchwald-HartwigCross-Coupling",
                "Chan_LamCoupling",
                "DielsAlder",
                "FischerIndoleSynthesis",
                "Friedel-CraftsAcylation",
                "Friedel-CraftsAlkylation",
                "GrignardReaction",
                "HiyamaCoupling",
                # "HuisgenCycloaddition",
                "Hydrogenation",
                "Kabachnik-FieldsReaction",
                "KumadaCoupling",
                "MannichReaction",
                "NegishiCoupling",
                "PausonKhandReaction",
                "ReductiveAmination",
                "SuzukiCoupling",
                "WittigReaction",
                ]

## Analysis of the reaction templates including the distinguishment of aromatic and aliphatic atoms

In [10]:
table = PrettyTable()
table.field_names = ["Reaction name", "No. of first templates", "No. of first data", "No. of second templates", "No. of second data", "No. of RDChrial templates"]
for rxn in all_rxn_class:
    num_frist_data = get_num_processed_data(f"../data/{rxn}/MappingResult_{rxn}.txt.processed")
    num_first_template = get_num_templates(f"../data/{rxn}/all_templates_used.csv")
    num_second_data = get_num_processed_data(f"../data_aro_ali_diff/{rxn}/MappingResult_{rxn}.txt.processed")
    num_second_template = get_num_templates(f"../data_aro_ali_diff/{rxn}/all_templates_used.csv")
    num_rdchiral_template = get_num_templates(f"../data/{rxn}/rdchiral_templates.csv")
    table.add_row([rxn, num_first_template, num_frist_data, num_second_template, num_second_data, num_rdchiral_template])

print(table)

+--------------------------------+------------------------+-------------------+-------------------------+--------------------+---------------------------+
|         Reaction name          | No. of first templates | No. of first data | No. of second templates | No. of second data | No. of RDChrial templates |
+--------------------------------+------------------------+-------------------+-------------------------+--------------------+---------------------------+
|      AdamsDecarboxylation      |           54           |        1641       |            55           |        1595        |            1491           |
|     Baylis-HillmanReaction     |           84           |        6101       |            87           |        6080        |            2697           |
| Buchwald-HartwigCross-Coupling |           96           |       16627       |           104           |       16545        |            4081           |
|        Chan_LamCoupling        |           43           |        634

### Visualized the reactions that using aromatic-aliphatic-distinguighing templates cannot recover.

In [6]:
import sys
sys.path.append("../")
from autotemplate.extract_utils import extract_from_rxn_smiles
from autotemplate.run_utils import rdchiralRunText_modified

rxn_smiles = "[nH:5]1[cH:6][n:7][c:8]2[cH:9][cH:10][cH:11][cH:12][c:13]12.c1ccc([B-](c2ccccc2)(c2ccccc2)[c:4]2[cH:3][cH:2][cH:1][cH:15][cH:14]2)cc1>>[cH:1]1[cH:2][cH:3][c:4](-[n:5]2[cH:6][n:7][c:8]3[cH:9][cH:10][cH:11][cH:12][c:13]23)[cH:14][cH:15]1"
product = rxn_smiles.split(">>")[1]
reaction_ids = "50699132"
generic_template = extract_from_rxn_smiles(rxn_smiles)
aromatic_aliphatic_distinguighing_template = "[c:1]-[n:2]>>[c:1]-[B-](-c1:c:c:c:c:c:1)(-c1:c:c:c:c:c:1)-c1:c:c:c:c:c:1.[n:2]"

print(rdchiralRunText_modified(generic_template, product))

['[cH:1]1[cH:2][cH:3][c:4]([B-:16]([c:17]2[cH:18][cH:19][cH:20][cH:21][cH:22]2)([c:23]2[cH:24][cH:25][cH:26][cH:27][cH:28]2)[c:29]2[cH:30][cH:31][cH:32][cH:33][cH:34]2)[cH:14][cH:15]1.[nH:5]1[cH:6][n:7][c:8]2[cH:9][cH:10][cH:11][cH:12][c:13]12']


In [149]:
from rdkit import Chem
mol = Chem.MolFromSmiles("[nH:5]1[cH:6][n:7][c:8]2[cH:9][cH:10][cH:11][cH:12][c:13]12")
Chem.Kekulize(mol)
Chem.MolToSmiles(mol, kekuleSmiles=True)

'[NH:5]1[CH:6]=[N:7][C:8]2=[CH:9][CH:10]=[CH:11][CH:12]=[C:13]12'

In [152]:
import CGRtools
from CGRtools.containers import ReactionContainer, MoleculeContainer

def cgr_representation(rxn_smiles):
    r = CGRtools.smiles(rxn_smiles)
    r.clean2d()
    cgr = ~r
    return cgr.__str__()

rxn_smiles = "Cl[C:1]([CH3:2])=[O:3].[CH3:4][c:5]1[cH:6][cH:7][cH:8][cH:9][c:10]1[OH:11]>>[C:1]([CH3:2])(=[O:3])[O:11][c:10]1[c:5]([CH3:4])[cH:6][cH:7][cH:8][cH:9]1"
rxn_smiles = "[nH:5]1[cH:6][n:7][c:8]2[cH:9][cH:10][cH:11][cH:12][c:13]12.c1ccc([B-](c2ccccc2)(c2ccccc2)[c:4]2[cH:3][cH:2][cH:1][cH:15][cH:14]2)cc1>>[cH:1]1[cH:2][cH:3][c:4](-[n:5]2[cH:6][n:7][c:8]3[cH:9][cH:10][cH:11][cH:12][c:13]23)[cH:14][cH:15]1"
r = CGRtools.smiles("[nH:5]1[cH:6][n:7][c:8]2[cH:9][cH:10][cH:11][cH:12][c:13]12.c1ccc([B-](c2ccccc2)(c2ccccc2)[c:4]2[cH:3][cH:2][cH:1][cH:15][cH:14]2)cc1")
# r.clean2d()
p = CGRtools.smiles("[cH:1]1[cH:2][cH:3][c:4](-[n:5]2[cH:6][n:7][c:8]3[cH:9][cH:10][cH:11][cH:12][c:13]23)[cH:14][cH:15]1")
# p.clean2d()

cgr = r^p
# cgr.clean2d()

decomposed = ReactionContainer.from_cgr(cgr)
# decomposed.clean2d()
decomposed
# decomposed.fix_mapping(logging=True)
m = decomposed.products[0]
# # print(m)
# reactant_part, product_part = cgr.decompose()
# print(product_part)
# product_part.


In [159]:
for a in m.atoms():
    print(a)
    break
af = a[1]
af

(1, C())


TypeError: expected 1 arguments, got 0

In [140]:
m

In [125]:
m = decomposed.reactants[0]
print(m.atoms_order)
m in [m, r]

{16: 1, 24: 1, 30: 1, 1: 1, 20: 2, 18: 3, 22: 3, 26: 3, 28: 3, 32: 3, 33: 3, 3: 3, 14: 3, 19: 4, 21: 4, 27: 4, 4: 4, 17: 5, 23: 5, 25: 5, 29: 5, 31: 5, 34: 5, 2: 5, 15: 5}


True

In [122]:
for atom in m.atoms():
    print(atom)
    a = atom[1]
    break
for bond in m.bonds():
    print(bond)
    break
print(a.charge)
print(a.atomic_symbol)

(16, C())
(16, 17, Bond(4))
0
C


In [117]:
b = bond[2]
b.order



14

In [115]:
from rdkit import Chem
def MolFromGraphs(node_list, adjacency_matrix, mapping_num_list, charge_list):
    mol = Chem.RWMol()
    node_to_idx = {}
    idx_to_mapping_num = {}
    idx_to_charge = {}
    for i in range(len(node_list)):
        a = Chem.Atom(node_list[i])
        molIdx = mol.AddAtom(a)
        node_to_idx[i] = molIdx
        idx_to_mapping_num[molIdx] = mapping_num_list[i]
        idx_to_charge[molIdx] = charge_list[i]

    # add bonds between adjacent atoms
    for ix, row in enumerate(adjacency_matrix):
        for iy, bond in enumerate(row):
            # only traverse half the matrix
            if iy <= ix:
                continue
            if bond == 0:
                continue
            elif bond == 1:
                bond_type = Chem.rdchem.BondType.SINGLE
                mol.AddBond(node_to_idx[ix], node_to_idx[iy], bond_type)
            elif bond == 2:
                bond_type = Chem.rdchem.BondType.DOUBLE
                mol.AddBond(node_to_idx[ix], node_to_idx[iy], bond_type)
            elif bond == 3:
                bond_type = Chem.rdchem.BondType.TRIPLE
                mol.AddBond(node_to_idx[ix], node_to_idx[iy], bond_type)
            elif bond == 4:
                bond_type = Chem.rdchem.BondType.AROMATIC
                mol.AddBond(node_to_idx[ix], node_to_idx[iy], bond_type)
    
    for molIdx, mapping_num in idx_to_mapping_num.items():
        atom = mol.GetAtomWithIdx(molIdx)
        atom.SetAtomMapNum(mapping_num)
        atom.SetFormalCharge(idx_to_charge[molIdx])
    mol = mol.GetMol()
    return mol

nodes = ["C", "C", "C", "C", "C", "C", "O", "N"]
mapping_num_list = [2, 3, 5, 7, 8, 9, 19, 21]
charge_list = [0,0,0,0,0,0,0,+1]
a = [[ 0.,  4.,  0.,  0.,  0.,  4.,  0.,  1.],
     [ 4.,  0.,  4.,  0.,  0.,  0.,  1.,  0.],
     [ 0.,  4.,  0.,  4.,  0.,  0.,  0.,  0.],
     [ 0.,  0.,  4.,  0.,  4.,  0.,  0.,  0.],
     [ 0.,  0.,  0.,  4.,  0.,  4.,  0.,  0.],
     [ 4.,  0.,  0.,  0.,  4.,  0.,  0.,  0.],
     [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
     [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]

test_mol = MolFromGraphs(nodes, a, mapping_num_list, charge_list)
Chem.MolToSmiles(test_mol)


'[c:2]1([NH3+:21])[c:3]([OH:19])[cH:5][cH:7][cH:8][cH:9]1'

## Analysis of downsampling

In [9]:
percents = [str(10), str(25), str(50), str(75)]
table = PrettyTable()
table.field_names = ["Reaction name", "No. templates (10%)", "Resiudal proportion (10%)", "No. templates (25%)", "Resiudal proportion (25%)", 
                     "No. templates (50%)", "Resiudal proportion (50%)", "No. templates (75%)", "Resiudal proportion (75%)", "No. templates (100%)", "Resiudal proportion (100%)"]
for rxn in all_rxn_class:
    table_row = [rxn]
    for percent in percents:
        rp_percent_data = get_residual_proportion(f"../data_{percent}perc/{rxn}/MappingResult_{rxn}.txt")
        num_percent_template = get_num_templates(f"../data_{percent}perc/{rxn}/all_templates_used.csv")
        table_row += [num_percent_template, rp_percent_data]
    rp_frist_data = get_residual_proportion(f"../data/{rxn}/MappingResult_{rxn}.txt")
    num_first_template = get_num_templates(f"../data/{rxn}/all_templates_used.csv")
    table_row += [num_first_template, rp_frist_data]
    table.add_row(table_row)
print(table)

+--------------------------------+---------------------+---------------------------+---------------------+---------------------------+---------------------+---------------------------+---------------------+---------------------------+----------------------+----------------------------+
|         Reaction name          | No. templates (10%) | Resiudal proportion (10%) | No. templates (25%) | Resiudal proportion (25%) | No. templates (50%) | Resiudal proportion (50%) | No. templates (75%) | Resiudal proportion (75%) | No. templates (100%) | Resiudal proportion (100%) |
+--------------------------------+---------------------+---------------------------+---------------------+---------------------------+---------------------+---------------------------+---------------------+---------------------------+----------------------+----------------------------+
|      AdamsDecarboxylation      |          5          |           37.26           |          9          |           42.79           |     